In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, random_split
import numpy as np
import pandas as pd
import math
import torch.nn.functional as F
import sys
import csv
from torch.nn.functional import normalize

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Network Construction

Construct multilayer neural network for the pain prediction dataset with input dimension of 22.

In [41]:
class InferenceBlock(nn.Module):
    def __init__(self, input_units, d_theta, output_units):
        """
        :param d_theta: dimensionality of the intermediate hidden layers.
        :param output_units: dimensionality of the output.
        :return: batch of outputs.
        """
        super(InferenceBlock, self).__init__()
        self.module = nn.Sequential(
            nn.Linear(input_units, d_theta, bias=True),
            nn.ELU(inplace=True),
            nn.Linear(input_units, d_theta, bias=True),
            nn.ELU(inplace=True),
            nn.Linear(d_theta, output_units, bias=True)
        )

    def forward(self, inps):
        out = self.module(inps)
        return out

class Amortized(nn.Module):
    def __init__(self, input_units=400, d_theta=400, output_units=400):
        super(Amortized, self).__init__()
        self.output_units = output_units
        self.weight_mean         = InferenceBlock(input_units, d_theta, output_units)
        self.weight_log_variance = InferenceBlock(input_units, d_theta, output_units)

    def forward(self, inps):
        weight_mean         = self.weight_mean(inps)
        weight_log_variance = self.weight_log_variance(inps)
        return weight_mean, weight_log_variance


class MLP(nn.Module):
    """
    Two layer MLP for pain prediction dataset benchmarks.
    """
    def __init__(self, hiddens, config):
        super(MLP, self).__init__()
        self.W1 = nn.Linear(22, hiddens)
        self.relu = nn.ReLU(inplace=True)
        self.dropout_1 = nn.Dropout(p=config['dropout'])
        self.W2 = nn.Linear(hiddens, hiddens)
        self.dropout_2 = nn.Dropout(p=config['dropout'])
        # self.W3 = nn.Linear(hiddens, 10)
        self.post  = Amortized(hiddens, hiddens, hiddens)
        self.prior = Amortized(hiddens, hiddens, hiddens)

    def forward(self, x, task_id=None, post = False, predefined = False):
        x = x.view(-1, 22)
        out = self.W1(x)
        out = self.relu(out)
        out = self.dropout_1(out)
        out = self.W2(out)
        out = self.relu(out)
        out = self.dropout_2(out)
        out_features = self.normalize(out)
        out_mean = torch.mean(out_features, dim=0, keepdim=True)
        if post:
            mu, logvar = self.post(out_mean)
        else:
            mu, logvar = self.prior(out_mean)
        return out_features, mu, logvar

    def normalize(self, x):
        max_val = x.max()
        min_val = x.min()
        return (x - min_val) / (max_val - min_val)
    
def get_pain_prediction(num_tasks):

    """
    Returns the dataset loaders for all tasks of pain prediction dataset
    """

    datasets = {}
    data_folder = r"C:\Users\ronny\Documents\GitHub\MetaPain\Data\biosignals_filtered_Processed\eda"
    files = os.listdir(data_folder)
    for task_id in range(1,num_tasks+1):

#         with open(os.path.join(data_folder, files[task_id-1]),'r') as f:
        with open(data_folder+'/'+ files[task_id-1],'r') as f:

            pickle_in = list(csv.reader(f, delimiter=','))
            feature,target = np.array(pickle_in)[:,:-1].astype(float),np.array(pickle_in)[:,-1]
            ind = ~np.isnan(feature).any(axis=1)
            feature = feature[ind, :]
            target = target[ind]
            target = np.unique(target, return_inverse=True)[1].astype(float)
            feature = normalize(torch.Tensor(feature),dim=0)
            dataset = TensorDataset(feature,torch.Tensor(target))
            train_dataset, test_dataset = random_split(dataset, [75, (len(target)-75)], generator=torch.Generator().manual_seed(42))
            datasets[task_id] = {'train': train_dataset, 'test': test_dataset}

    return datasets
    
def get_benchmark_model(hiddens, dropout):
    """
    Return the corresponding PyTorch model for experiment
    :param args:
    :return:
    """
    return MLP(hiddens, {'dropout': dropout}).to(DEVICE)

def get_benchmark_dataset(dataset):
    """
    Returns the benchmark loader which could be either of these:
    get_split_cifar100_tasks, get_permuted_mnist_tasks, or get_rotated_mnist_tasks

    :param args:
    :return: a function which when called, returns all tasks
    """
    if dataset == "pain_prediction":
        return get_pain_prediction
    else:
        raise ValueError("this function can only get pain_prediction dataset")

In [42]:
EXPERIMENT_DIRECTORY = r"C:\Users\ronny\Documents\GitHub\MetaPain\simulation"

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def iterate_data(loader):
    xs,ys = [], []
    for x,y in loader:
        xs.append(x)
        ys.append(y)
    xs = torch.cat(xs, dim=0).squeeze(1)
    ys = torch.cat(ys, dim=0)
    return xs, ys


def sample(mu, logvar, L, device):
    shape = (L, ) + mu.size()
    eps   = torch.randn(shape).to(device)
    w     = mu.unsqueeze(0) + eps * logvar.exp().sqrt().unsqueeze(0)
    return w

def rand_features(bases, features, bias):
    # tf.random_normal()
    return math.sqrt(2/bias.shape[0]) * torch.cos(torch.matmul(bases, features) + bias)

def dotp_kernel(feature1, feature2):
    return torch.matmul(feature1, feature2)

def rbf_kernel(feature1, feature2):
    res = torch.exp(-0.25 * torch.norm(feature1.unsqueeze(1)-feature2, dim=2, p=1))
    return res

def linear(feature1, feature2):
    return feature1 @ feature2.T

def poly(feature1, feature2):
    return (torch.matmul(feature1, feature2.T) + 1).pow(3)

def kl_div(m, log_v, m0, log_v0):

    v = log_v.exp()
    v0 = log_v0.exp()

    dout, din = m.shape
    const_term = -0.5 * dout * din

    log_std_diff = 0.5 * torch.sum(torch.log(v0) - torch.log(v))
    mu_diff_term = 0.5 * torch.sum((v + (m0-m)**2) / v0)
    kl = const_term + log_std_diff + mu_diff_term
    return kl

def cosine_dist(a, b):
    # sqrt(<a, b>) / (sqrt(<a, a>), sqrt(<b, b>))
    a = a.view(1, -1)
    b = b.view(1, -1)
    normalize_a = F.normalize(a, dim=-1)# %
    normalize_b = F.normalize(b, dim=-1)
    return torch.sqrt(torch.sum(torch.multiply(normalize_a, normalize_b)))


def get_train_loader(dataset, batch_size):
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, num_workers=8, pin_memory=True, shuffle=True)
    return train_loader


def get_test_loader(dataset):
    test_loader  = torch.utils.data.DataLoader(dataset,  batch_size=5, shuffle=False, num_workers=3, pin_memory=True)
    return test_loader

def log_metrics(metrics, time, prev_task_id, current_task_id, acc_db, loss_db):
    """
    Log accuracy and loss at different times of training
    """
    print('epoch {}, task:{}, metrics: {}'.format(time, prev_task_id, metrics))
    # log to db
    acc = metrics['accuracy']
    loss = metrics['loss']
    loss_db[prev_task_id][current_task_id-1]= loss
    acc_db[prev_task_id][current_task_id-1] = acc
    return acc_db, loss_db


def init_experiment(tasks):
    print('------------------- Experiment started -----------------')
    # print(f"Parameters:\n  seed={args.seed}\n  benchmark={args.dataset}\n  num_tasks={args.tasks}\n  "+
    # 	  f"epochs_per_task={args.epochs_per_task}\n  batch_size={args.batch_size}\n  "+
    # 	  f"learning_rate={args.lr}\n  learning rate decay(gamma)={args.gamma}\n  dropout prob={args.dropout}\n")

    # # 1. setup seed for reproducibility
    # random.seed(args.seed)
    # np.random.seed(args.seed)
    # torch.manual_seed(args.seed)
    # torch.cuda.manual_seed(args.seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    # # torch.backends.cudnn.enabled = False

    # # 2. create directory to save results
    # Path(EXPERIMENT_DIRECTORY).mkdir(parents=True, exist_ok=True)
    # print("The results will be saved in {}\n".format(EXPERIMENT_DIRECTORY))

    # 3. create data structures to store metrics
    # loss_db = {t: [0 for i in range(args.tasks*args.epochs_per_task)] for t in range(1, args.tasks+1)}
    # acc_db =  {t: [0 for i in range(args.tasks*args.epochs_per_task)] for t in range(1, args.tasks+1)}
    loss_db = {t: [0 for i in range(tasks)] for t in range(1, tasks+1)}
    acc_db =  {t: [0 for i in range(tasks)] for t in range(1, tasks+1)}
    return acc_db, loss_db


def end_experiment(acc_db,d_rn_f):

    # 1. save all metrics into csv file
    acc_df = pd.DataFrame(acc_db)
    acc_df.to_csv(EXPERIMENT_DIRECTORY+'/eda_accs.csv')


    # 2. calculate average accuracy and forgetting (c.f. ``evaluation`` section in our paper)
    score = np.mean([acc_db[i][-1] for i in acc_db.keys()])

    original_stdout = sys.stdout
    with open(EXPERIMENT_DIRECTORY+'/final_result_eda'+str(d_rn_f)+'.txt', 'w') as f:
        sys.stdout = f
        print('average accuracy = {}'.format(score))
        sys.stdout = original_stdout
    print('average accuracy = {}'.format(score))
    print()
    print('------------------- Experiment ended -----------------')

def coreset_selection(dataset, coreset_size, dataset_name):
    # Uniformly sample and select data per each class from (x_train, y_train) and add to current coreset (x_coreset, y_coreset)
    if 'mnist' in dataset_name:
        targets = dataset.targets
        sample_per_each_class = coreset_size // len(targets.unique())
    else:
        loader = get_test_loader(dataset)
        inputs, targets = iterate_data(loader)
        sample_per_each_class = coreset_size


    idx = []
    for cl in targets.unique():
        cl = cl.item()
        cls_idx = torch.where(targets == cl)[0]
        if len(cls_idx) == 0:
            continue
        cls_idx = cls_idx[torch.randperm(len(cls_idx))[:sample_per_each_class]]
        idx = idx + cls_idx.tolist()

#     remained_item = list(set(torch.arange(0, len(targets)).tolist()))
    remained_item = list(set(torch.arange(0, len(targets)).tolist()) - set(idx))
    if 'mnist' in dataset_name:
        train_dataset    = torch.utils.data.dataset.Subset(dataset, remained_item)
        coreset_dataset  = torch.utils.data.dataset.Subset(dataset, idx)
    else:
        train_dataset    = torch.utils.data.dataset.TensorDataset(inputs[remained_item], targets[remained_item])
        coreset_dataset  = torch.utils.data.dataset.TensorDataset(inputs[idx], targets[idx])
    return train_dataset, coreset_dataset

In [43]:
def train_single_epoch(net, optimizer, loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, task_id=None):
    """
    Train the model for a single epoch

    :param net:
    :param optimizer:
    :param loader:
    :param criterion:
    :param task_id:
    :return:
    """
    net = net.to(DEVICE)
    net.train()

    coreset_input, coreset_target = iterate_data(coreset_loader)
    coreset_input  = coreset_input.to(DEVICE)
#     coreset_target = torch.nn.functional.one_hot(coreset_target, num_classes=5)
    coreset_target = coreset_target.to(DEVICE).float()

    task_acc = AverageMeter()
    task_loss = AverageMeter()

    # import pdb; pdb.set_trace()
    for _, (data, target) in enumerate(loader):
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        batch_size = data.shape[0]
        optimizer.zero_grad()

        if task_id:
            features_train,   p_mu, p_log_var = net(data, task_id, post = False, predefined=True)
            features_coreset, r_mu, r_log_var  = net(coreset_input, task_id, post = True)
        else:
            features_train,   p_mu, p_log_var = net(data,          post = False, predefined=True)
            features_coreset, r_mu, r_log_var  = net(coreset_input, post = True)

        # compute_bases
        rs = sample(r_mu, r_log_var, d_rn_f, DEVICE).squeeze()


        # Here we map the coreset data into random fourier space. 
        x_supp_phi_t = rand_features(rs, torch.transpose(features_coreset, 1, 0), bias)  # (d_rn_f , number of coreset samples)
        # in the random fourier space we compute the the kernel matrix, the dot product between mapped samples
        support_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_supp_phi_t)  # (number of coreset samples , number of coreset samples)

        # closed-form solution with trainable lmd
        # alpha = torch.matmul(torch.inverse(support_kernel + (lmd_abs + 0.01) * torch.eye(support_kernel[0])), target)
        # (number of coreset samples, number of classes)
        alpha = torch.matmul(torch.inverse(support_kernel + (torch.abs(lmd[task_id-1]) + 0.01) * torch.eye(support_kernel.shape[0]).to(DEVICE)), coreset_target) 

        # Here we map the train data into random fourier space. 
        x_que_phi_t = rand_features(rs, torch.transpose(features_train, 1, 0), bias)

        # Compute cross kernel
        cross_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_que_phi_t) # (number of coreset samples, number of train samples)

        pred = gma[task_id-1] * torch.matmul(cross_kernel.T, alpha) + bta[task_id-1]

        kl_loss = kl_div(r_mu, r_log_var, p_mu, p_log_var)

        loss = criterion(pred, target) + tau * kl_loss
        task_loss.update(loss.item())

        acc = (pred.argmax(dim=-1) == target).float().mean()
        task_acc.update(acc.item())


        loss.backward()
        optimizer.step()

    print(task_acc.avg, task_loss.avg)
    return net

In [44]:
def eval_single_epoch(net, loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, task_id=None):
    """
    Evaluate the model for single epoch

    :param net:
    :param loader:
    :param criterion:
    :param task_id:
    :return:
    """

    net = net.to(DEVICE)
    net.eval()

    coreset_input, coreset_target = iterate_data(coreset_loader)
    coreset_input  = coreset_input.to(DEVICE)
#     coreset_target = torch.nn.functional.one_hot(coreset_target, num_classes=5)
    coreset_target = coreset_target.to(DEVICE).float()

    test_loss = AverageMeter()
    correct   = AverageMeter()

    with torch.no_grad():
        for data, target in loader:
            data = data.to(DEVICE)
            target = target.to(DEVICE)
            if task_id:
                features_test,   p_mu, p_log_var = net(data, task_id, post = False)
                features_coreset, r_mu, r_log_var  = net(coreset_input, task_id, post = True)
            else:
                features_test,   p_mu, p_log_var = net(data,          post = False)
                features_coreset, r_mu, r_log_var  = net(coreset_input, post = True)

            # compute_bases
            rs = sample(r_mu, r_log_var, d_rn_f, DEVICE).squeeze()

            x_supp_phi_t = rand_features(rs, torch.transpose(features_coreset, 1, 0), bias)  

            support_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_supp_phi_t)  

            # closed-form solution with trainable lmd
            alpha = torch.matmul(torch.inverse(support_kernel + (torch.abs(lmd[task_id-1]) + 0.01) * torch.eye(support_kernel.shape[0]).to(DEVICE)), coreset_target)
            x_que_phi_t = rand_features(rs, torch.transpose(features_test, 1, 0), bias)

            # Compute cross kernel
            cross_kernel = dotp_kernel(torch.transpose(x_supp_phi_t, 1, 0), x_que_phi_t)

            pred = gma[task_id-1] * torch.matmul(cross_kernel.T, alpha) + bta[task_id-1]
            # align loss
            
            loss = criterion(pred, target)

            test_loss.update(loss.item())

            acc = (pred.argmax(dim=-1) == target).float().mean()
            correct.update(acc.item())

    return {'accuracy': correct.avg, 'loss': test_loss.avg}

In [45]:
def run(num_tasks, hiddens, d_rn_f, epochs_per_task, batch_size, core_size, model_name, tau = 0.0001):
    """
    Run a single run of experiment.

    :param args: please see `utils.py` for arguments and options
    """
    

    # pdb.set_trace()
    acc_db, loss_db = init_experiment(num_tasks)
    print("Loading {} tasks for pain prediction".format(num_tasks))
    tasks = get_benchmark_dataset("pain_prediction")(num_tasks)
    print("loaded all tasks!")
    model = get_benchmark_model(hiddens, dropout=0.3)

    # criterion
    criterion = nn.MSELoss().to(DEVICE)
    bias = 2 * math.pi * torch.rand(d_rn_f, 1).to(DEVICE)
    time = 0

    # Learnable coefficient

    lmd   = torch.tensor([0.5      for _ in range(num_tasks)], requires_grad=True, device=DEVICE)
    gma   = torch.tensor([1.0      for _ in range(num_tasks)], requires_grad=True, device=DEVICE)
    bta   = torch.tensor([0.       for _ in range(num_tasks)], requires_grad=True, device=DEVICE)

    coreset_all = []
    # basis = []
    # rff = []
    # mu = []
    # log_var = []
    for current_task_id in range(1, num_tasks+1):
        print("================== TASK {} / {} =================".format(current_task_id, num_tasks))
        train_dataset = tasks[current_task_id]['train']
        train_dataset, coreset_dataset = coreset_selection(train_dataset, core_size, dataset_name = "pain_prediction")

        # import pdb; pdb.set_trace()
        # save coreset
        coreset_all.append(coreset_dataset)

        # uncomment for replay
        # coreset_dataset = torch.utils.data.dataset.ConcatDataset(coreset_all)
        # print(len(coreset_dataset))

        train_loader    = get_train_loader(train_dataset, batch_size)
        coreset_loader  = get_train_loader(coreset_dataset, batch_size)
        lr = max(0.1 * 0.5 ** (current_task_id), 0.00005) 
        for epoch in range(1, epochs_per_task+1):
            # 1. train and save
            params = list(model.parameters()) + [lmd, gma, bta]
            optimizer = torch.optim.SGD(params, lr=lr, momentum=0.8) 
#             optimizer = torch.optim.Adam(params, lr=lr)
            train_single_epoch(model, optimizer, train_loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, current_task_id)
            time += 1
            # import pdb; pdb.set_trace()
            # 2. evaluate on all tasks up to now, including the current task
            for prev_task_id in range(1, current_task_id+1):
                # 2.0. only evaluate once a task is finished
                if epoch == epochs_per_task:
                    model = model.to(DEVICE)
                    val_dataset = tasks[prev_task_id]['test']
                    coreset_dataset = coreset_all[prev_task_id-1]

                    val_loader      = get_test_loader(val_dataset)
                    coreset_loader  = get_test_loader(coreset_dataset)

                    # 2.1. compute accuracy and loss
                    metrics = eval_single_epoch(model, val_loader, coreset_loader, criterion, bias, lmd, gma, bta, d_rn_f, tau, current_task_id)
                    acc_db, loss_db = log_metrics(metrics, time, prev_task_id, current_task_id, acc_db, loss_db)

#                     # 2.3. save model parameters
#                     save_checkpoint(model, time)
        # basis, rff, mu, log_var = save_stats_one_task(model, coreset_all, bias, args, 1, basis, rff, mu, log_var)
    torch.save(model.state_dict(), EXPERIMENT_DIRECTORY+'/'+model_name)
    end_experiment(loss_db,d_rn_f)

In [46]:
run(num_tasks = 86, hiddens=64, d_rn_f = 128, epochs_per_task = 30, batch_size = 10, core_size = 6,model_name = 'MSE_128_eda', tau = 1)

------------------- Experiment started -----------------
Loading 86 tasks for pain prediction
loaded all tasks!
================== TASK 1 / 86 =================
0.1600000023841858 3.2428112983703614
0.0 2.6803142786026
0.04000000059604645 2.228584122657776
0.06000000089406967 2.062542939186096
0.1600000038743019 2.0422027587890623
0.04000000059604645 2.9625020027160645
0.0 2.4463048219680785
0.0800000011920929 2.6865880489349365
0.18000000417232515 2.2220948934555054
0.20000000596046447 2.236763334274292
0.0800000011920929 1.9638705253601074
0.12000000029802323 2.3825846195220945
0.20000000596046447 2.255341148376465
0.04000000059604645 2.2422393560409546
0.12000000327825547 2.3383216857910156
0.20000000298023224 2.3903582096099854
0.12000000178813934 2.041430640220642
0.12000000178813934 2.096444034576416
0.04000000059604645 2.044959545135498
0.2200000047683716 1.89963036775589
0.12000000327825547 1.831433606147766
0.10000000149011612 1.5759940981864928
0.12000000476837158 1.534291005

0.18000000417232515 1.7017377734184265
0.18000000566244126 1.9142993211746215
0.1600000038743019 1.6397390007972716
0.0800000011920929 1.935104739665985
0.06000000089406967 1.968937087059021
0.04000000059604645 1.974555003643036
0.06000000089406967 2.196111965179443
0.0800000011920929 1.9819450616836547
0.10000000298023223 1.889007043838501
epoch 180, task:1, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.165751004219055}
epoch 180, task:2, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.0042041659355163}
epoch 180, task:3, metrics: {'accuracy': 0.20000000298023224, 'loss': 2.13139750957489}
epoch 180, task:4, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.756307101249695}
epoch 180, task:5, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.9484904766082765}
epoch 180, task:6, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.9027361512184142}
================== TASK 7 / 86 =================
0.0800000011920929 2.393944430351257
0.1600000038743019 1.9514992475509643
0

epoch 300, task:6, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.83369323015213}
epoch 300, task:7, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.391008484363556}
epoch 300, task:8, metrics: {'accuracy': 0.21000000238418579, 'loss': 2.1957462787628175}
epoch 300, task:9, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.1603803277015685}
epoch 300, task:10, metrics: {'accuracy': 0.1300000011920929, 'loss': 2.006193518638611}
================== TASK 11 / 86 =================
0.12000000327825547 1.6517377853393556
0.10000000298023223 1.8611363530158997
0.16000000536441802 1.8311264038085937
0.10000000298023223 1.961941373348236
0.14000000357627868 1.9394238591194153
0.12000000178813934 1.8667257308959961
0.18000000715255737 1.867263126373291
0.12000000178813934 1.5661540508270264
0.020000000298023225 1.936200785636902
0.20000000149011612 1.9084629654884337
0.0800000011920929 2.286454808712006
0.12000000178813934 1.8254417657852173
0.16000000536441802 2.096116805076599
0.1

0.10000000149011612 1.4583202481269837
0.18000000417232515 1.697043240070343
0.14000000357627868 1.828851842880249
0.10000000298023223 1.7550208806991576
0.14000000208616256 1.9617063283920289
epoch 420, task:1, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.1308024406433106}
epoch 420, task:2, metrics: {'accuracy': 0.1600000023841858, 'loss': 0.9687891364097595}
epoch 420, task:3, metrics: {'accuracy': 0.20000000298023224, 'loss': 2.033252239227295}
epoch 420, task:4, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.6670720338821412}
epoch 420, task:5, metrics: {'accuracy': 0.2800000041723251, 'loss': 1.863048791885376}
epoch 420, task:6, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7798189640045166}
epoch 420, task:7, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.364374542236328}
epoch 420, task:8, metrics: {'accuracy': 0.21000000238418579, 'loss': 2.1917418003082276}
epoch 420, task:9, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.1369231700897218}
epoc

epoch 510, task:6, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.8191230177879334}
epoch 510, task:7, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.3686289191246033}
epoch 510, task:8, metrics: {'accuracy': 0.21000000238418579, 'loss': 1.8921708822250367}
epoch 510, task:9, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.138737016916275}
epoch 510, task:10, metrics: {'accuracy': 0.1300000011920929, 'loss': 2.0479985952377318}
epoch 510, task:11, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.314621740579605}
epoch 510, task:12, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.933365273475647}
epoch 510, task:13, metrics: {'accuracy': 0.24000000357627868, 'loss': 0.8669968783855438}
epoch 510, task:14, metrics: {'accuracy': 0.1300000011920929, 'loss': 1.3271088123321533}
epoch 510, task:15, metrics: {'accuracy': 0.0800000011920929, 'loss': 1.583964455127716}
epoch 510, task:16, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.7729628086090088}
epoch 510,

...

epoch 2160, task:48, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.3936449408531189}
epoch 2160, task:49, metrics: {'accuracy': 0.12000000178813934, 'loss': 2.2303441762924194}
epoch 2160, task:50, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.439676034450531}
epoch 2160, task:51, metrics: {'accuracy': 0.3700000047683716, 'loss': 1.9957124710083007}
epoch 2160, task:52, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.5645123720169067}
epoch 2160, task:53, metrics: {'accuracy': 0.1600000023841858, 'loss': 2.151589035987854}
epoch 2160, task:54, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.8739873886108398}
epoch 2160, task:55, metrics: {'accuracy': 0.28000000715255735, 'loss': 2.003541898727417}
epoch 2160, task:56, metrics: {'accuracy': 0.24000000357627868, 'loss': 2.176072359085083}
epoch 2160, task:57, metrics: {'accuracy': 0.2800000041723251, 'loss': 2.2444398641586303}
epoch 2160, task:58, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.7604696393013}


epoch 2190, task:54, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.875976037979126}
epoch 2190, task:55, metrics: {'accuracy': 0.28000000715255735, 'loss': 2.040631723403931}
epoch 2190, task:56, metrics: {'accuracy': 0.24000000357627868, 'loss': 2.169300079345703}
epoch 2190, task:57, metrics: {'accuracy': 0.2800000041723251, 'loss': 2.247958278656006}
epoch 2190, task:58, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.766943633556366}
epoch 2190, task:59, metrics: {'accuracy': 0.24000000357627868, 'loss': 1.7969914197921752}
epoch 2190, task:60, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.4870346069335938}
epoch 2190, task:61, metrics: {'accuracy': 0.17000000178813934, 'loss': 1.9896944522857667}
epoch 2190, task:62, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.2995935738086701}
epoch 2190, task:63, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.3388980627059937}
epoch 2190, task:64, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.9007397413253784

epoch 2220, task:59, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.7528865814208985}
epoch 2220, task:60, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.483290195465088}
epoch 2220, task:61, metrics: {'accuracy': 0.17000000178813934, 'loss': 2.1113956928253175}
epoch 2220, task:62, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.287530142068863}
epoch 2220, task:63, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.4374560713768005}
epoch 2220, task:64, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.9202858209609985}
epoch 2220, task:65, metrics: {'accuracy': 0.04000000059604645, 'loss': 1.864565920829773}
epoch 2220, task:66, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.8424751996994018}
epoch 2220, task:67, metrics: {'accuracy': 0.17000000178813934, 'loss': 2.1984906911849977}
epoch 2220, task:68, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.791788673400879}
epoch 2220, task:69, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.89190578460693

epoch 2250, task:63, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.3931716918945312}
epoch 2250, task:64, metrics: {'accuracy': 0.20000000298023224, 'loss': 1.9209336042404175}
epoch 2250, task:65, metrics: {'accuracy': 0.0800000011920929, 'loss': 1.8135775089263917}
epoch 2250, task:66, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.8382381200790405}
epoch 2250, task:67, metrics: {'accuracy': 0.17000000178813934, 'loss': 2.2574567079544066}
epoch 2250, task:68, metrics: {'accuracy': 0.0800000011920929, 'loss': 1.7833947896957398}
epoch 2250, task:69, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.8821651935577393}
epoch 2250, task:70, metrics: {'accuracy': 0.0800000011920929, 'loss': 1.8324049711227417}
epoch 2250, task:71, metrics: {'accuracy': 0.1600000023841858, 'loss': 1.1692058801651002}
epoch 2250, task:72, metrics: {'accuracy': 0.12000000178813934, 'loss': 1.384538221359253}
epoch 2250, task:73, metrics: {'accuracy': 0.20000000298023224, 'loss': 2.0428321957588

KeyboardInterrupt: 